In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
from package.yfdownloader import get_ohlc
pio.renderers.default = 'notebook'


def run(ticker, benchmark='^SPX', window=90):
    df = get_ohlc(ticker)['Close'].pct_change().dropna()
    bench = get_ohlc(benchmark)['Close'].pct_change().dropna()
    
    betas = []
    dates = df.index[window-1:]
    
    for i in range(len(df) - window + 1):
        df_window = df.iloc[i:i+window]
        bench_window = bench.iloc[i:i+window]
        cov_matrix = np.cov(df_window, bench_window)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        betas.append(beta)
    
    beta_series = pd.Series(betas, index=dates)
    
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=beta_series.index,
            y=beta_series,
            name=f'{ticker} Rolling Beta ({window}d)',
            line=dict(color='lightgrey',dash='dot',shape='spline'),
            hovertemplate='<b>%{x}</b><br>%{y:.2f}'
        )
    )
    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Beta',
        title=dict(text=f'Rolling Beta of {ticker} vs {benchmark}',font=dict(family='monospace',size=17),x=0.5),
        template='plotly_dark',
        font=dict(size=11)
    )
    fig.show()
    

run('BLK', window=90)